In [1]:
#!/usr/bin/env python
# coding: utf-8

from fastai.text import *
import fastai
from pathlib import Path
import numpy as np, pandas as pd
import chardet
import random
import torch
import logging

class py_mincor_cls:
    def __init__(self, path):
        nm_dataset = 'Enron_emails'        
        self.path = Path(path)
        self.data_clas = load_data(self.path.joinpath("tmp"), 'data_clas.pkl', bs=12)
        self.learn = text_classifier_learner(self.data_clas, AWD_LSTM, drop_mult=0.5)
        self.learn.load('fourth')
        self.emailAddr = ""
        self.lbl = ['Company Business', 'Purely Personal', 'Personal but in professional context', 'Logistic Arrangements', 'Employment arrangements', 'Document editing', 'Empty']
        self.df_client = pd.read_csv(self.path.joinpath(nm_dataset).joinpath("emails.csv"))
        
    def assignto_setEmailAddr(self, emailAddr):
        self.emailAddr = emailAddr
    def assignto_predict(self, txt):
        return (self.learn.predict(self.assignto_preprocessTxt(txt)))
    def assignto_preprocessTxt(self, text):       
        return(text)
    def assignto_getProbability(self, txt):
        return (self.assignto_predict(txt)[2].numpy())
    def assignto_getProbLab(self, txt):
        return (pd.DataFrame({"label": self.lbl, "prob": self.assignto_getProbability(txt)}))
    
    def contacts_getEmailAddrFromText(self, text):
        # @ for as in the Email 
        # + for Repeats a character one or more times 
        lst = re.findall('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', text)      
        # Printing of List 
        return([e.upper() for e in list(set(lst))])

    def contacts_getFoundEmailList(self, text): 
        return(self.df_client[self.df_client['EMAIL'].str.match(pat = '|'.join(self.contacts_getEmailAddrFromText(text)))])

    def contacts_getFoundEmailStr(self, text): 
        df = self.contacts_getFoundEmailList(text)
        str_out = list(df['FNAME'] + " | " + df['LNAME'] + " | " + df['EMAIL'])
        return(str_out)

    def tools_delete_token_sent(self, s, token):
        return(re.sub(r".*" + token + ":.*", "\n", s))

    def extractEmailBody(self, text):
        s = self.tools_delete_token_sent(text, "From")
        s = self.tools_delete_token_sent(text, "Sent")
        s = self.tools_delete_token_sent(s, "To")
        s = self.tools_delete_token_sent(s, "[Cc][Cc]")
        s = self.tools_delete_token_sent(s, "Subject")
        return(s)        
        
# interface to the world    
m1 = py_mincor_cls("/home/wk/myProjects/Correspondance/data")


In [30]:

# Python program to extract emails From  
# the String By Regular Expression. 
  
# Importing module required for regular 
# expressions 
import re  
  
# Example string  
s = '''
-----Original Message-----
From: 	Pam.Allison@dynegy.com@ENRON [mailto:IMCEANOTES-Pam+2EAllison+40dynegy+2Ecom+40ENRON@ENRON.com]  On Behalf Of Pamela.J.Allison@dynegy.com
Sent:	Wednesday, August 15, 2001 5:39 PM
To:	Lay, Kenneth
Subject:	Jeff's Leaving

20relations@caiso.com

Mr. Lay:

Mr. Lay, the I am not writing this in malice but in hopes that it helps get
Enron back the way it used to treat their employees and makes it the number
one employer of choice again.  I hope you can get back the feeling that I
had when I first started there and get the stress level down in your
organization for the sake of your employees.

I am a former employee who worked at Enron for 10 years - my husband was,
at one time, in charge of A/P.  My one big disappointment was that I never
got to meet you.    During that time, I made a lot of friends who I
continue to see from time to time.  I left last year and have missed the
company since I left but would not contemplate coming back unless several
people have been replaced.  (Joannie Williamson and Sheri Sera know my
story.)

I "knew" Jeff but I don't think he knew me by name, only by sight, even
though I worked in ENA when it only had 400 employees with 3 of us in HR.
He was NOT what I would call a people person!  Unless you were in an upper
level position, he did not take the time to find out your name.  (Ron Burns
is still one of my favorite people because of his talent with people - we
need more like him in the business world!)

As you can see, I am now working for a competitor and since I work in HR, I
continually run into former Enron workers who have also left - most of
their reasons are the same.

It is not that they have lost faith in Enron as a company but because of
they way they were treated by their managers.  During the last 5 years I
was there, I noticed a change in direction in the way employees were
treated by upper management - and upper management was getting away with
it.  Not only were they getting away with it, these people were being
rewarded for this behavior.  I have heard stories of lower level employees
being screamed at and in one instance, one of the VP's who was brought down
from Canada was heard in his office screaming and pounding his telephone on
his desk.  Heaven only knows how he treats his subordinates.  Believe me,
the way employees are treated at Enron is being talked about on the streets
of Houston and on the different college campuses.  You might also at some
point take the time to find out why so many good people have left the HR
community at Enron and it is not because they wanted to.

I don't know if you will see this, but I hope so.  Good luck on bringing
Enron back to way it was - I still own stock!!

Thanks for listening...

Pam Allison 

'''
# @ for as in the Email 
# + for Repeats a character one or more times 
lst = re.findall('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', s)      
  
# Printing of List 
list(set(lst))


['Pamela.J.Allison@dynegy.com',
 '20relations@caiso.com',
 'IMCEANOTES-Pam+2EAllison+40dynegy+2Ecom+40ENRON@ENRON.com',
 'Pam.Allison@dynegy.com']

In [31]:
m1.extractEmailBody(s)

'\n-----Original Message-----\nFrom: \tPam.Allison@dynegy.com@ENRON [mailto:IMCEANOTES-Pam+2EAllison+40dynegy+2Ecom+40ENRON@ENRON.com]  On Behalf Of Pamela.J.Allison@dynegy.com\n\n\n\n\n\n\n\n20relations@caiso.com\n\nMr. Lay:\n\nMr. Lay, the I am not writing this in malice but in hopes that it helps get\nEnron back the way it used to treat their employees and makes it the number\none employer of choice again.  I hope you can get back the feeling that I\nhad when I first started there and get the stress level down in your\norganization for the sake of your employees.\n\nI am a former employee who worked at Enron for 10 years - my husband was,\nat one time, in charge of A/P.  My one big disappointment was that I never\ngot to meet you.    During that time, I made a lot of friends who I\ncontinue to see from time to time.  I left last year and have missed the\ncompany since I left but would not contemplate coming back unless several\npeople have been replaced.  (Joannie Williamson and She

In [32]:
import re
token ="From"
line = re.sub(r"^[\s]*" + token + ":.*", "______", s)
token ="Sent"
line = re.sub(r"^[ ]*" + token + ":.*", "______", line)
token ="To"
line = re.sub(r"^[\s]*" + token + ":.*", "______", line)
token ="Subject"
line = re.sub(r"^[\s]*" + token + ":.*", "______", line)

In [33]:
token ="From"
line = re.sub(r".*" + token + ":.*", "______", s)
token ="Sent"
line = re.sub(r".*" + token + ":.*", "______", line)
token ="To"
line = re.sub(r".*" + token + ":.*", "______", line)
token ="[Cc][Cc]"
line = re.sub(r".*" + token + ":.*", "______", line)
token ="Subject"
line = re.sub(r".*" + token + ":.*", "______", line)
line

'\n-----Original Message-----\n______\n______\n______\n______\n\n20relations@caiso.com\n\nMr. Lay:\n\nMr. Lay, the I am not writing this in malice but in hopes that it helps get\nEnron back the way it used to treat their employees and makes it the number\none employer of choice again.  I hope you can get back the feeling that I\nhad when I first started there and get the stress level down in your\norganization for the sake of your employees.\n\nI am a former employee who worked at Enron for 10 years - my husband was,\nat one time, in charge of A/P.  My one big disappointment was that I never\ngot to meet you.    During that time, I made a lot of friends who I\ncontinue to see from time to time.  I left last year and have missed the\ncompany since I left but would not contemplate coming back unless several\npeople have been replaced.  (Joannie Williamson and Sheri Sera know my\nstory.)\n\nI "knew" Jeff but I don\'t think he knew me by name, only by sight, even\nthough I worked in ENA wh

In [9]:
??load_data


In [34]:
m1.contacts_getFoundEmailList(s)

,Unnamed: 0,master_rootid,DOCKET,SUBJECT,LEAD,RECEIVED,PRIORITY,ASGNTO,FNAME,LNAME,...,EMAIL,TEL1,ADOC_REF,DESCRIPTION,FILESUFFIX,TRECS_Added_Date,filename,text,replyfname,reply


In [10]:
 	
r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"

'(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$)'

In [35]:
def getEmailAddrFromText(text):
    # @ for as in the Email 
    # + for Repeats a character one or more times 
    lst = re.findall('[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', s)      
    # Printing of List 
    return([e.upper() for e in list(set(lst))])

getEmailAddrFromText(s)

['PAMELA.J.ALLISON@DYNEGY.COM',
 '20RELATIONS@CAISO.COM',
 'IMCEANOTES-PAM+2EALLISON+40DYNEGY+2ECOM+40ENRON@ENRON.COM',
 'PAM.ALLISON@DYNEGY.COM']

In [37]:
df = pd.read_csv("/home/wk/myProjects/Correspondance/data/Enron_emails/contacts.csv")

In [38]:
df.tail()

,DetailID,FNAME,LNAME,PROV,EMAIL,TEL1,ORGANIZATION
87468,87470,NaN,NaN,NaN,MCCREEM@SCE.COM,NaN,SCE
87469,87471,RLUCK,DWP CI LA,NaN,RLUCK@DWP.CI.LA.CA.US,NaN,CA
87470,87472,NaN,NaN,NaN,SCE2@CAISO.COM,NaN,CAISO
87471,87473,NaN,NaN,NaN,BRAUN@BRAUNLEGAL.COM,NaN,BRAUNLEGAL
87472,87474,NaN,NaN,NaN,20RELATIONS@CAISO.COM,NaN,CAISO


In [43]:
result = df['EMAIL'].str.match(pat = '|'.join(getEmailAddrFromText(s)))
result

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
87443    False
87444    False
87445    False
87446    False
87447    False
87448    False
87449    False
87450    False
87451    False
87452    False
87453    False
87454    False
87455    False
87456    False
87457    False
87458    False
87459    False
87460    False
87461    False
87462    False
87463    False
87464    False
87465    False
87466    False
87467    False
87468    False
87469    False
87470    False
87471    False
87472     True
Name: EMAIL, Length: 87473, dtype: bool

In [42]:
'|'.join(getEmailAddrFromText(s))

'PAMELA.J.ALLISON@DYNEGY.COM|20RELATIONS@CAISO.COM|IMCEANOTES-PAM+2EALLISON+40DYNEGY+2ECOM+40ENRON@ENRON.COM|PAM.ALLISON@DYNEGY.COM'

In [44]:
df[result]

,DetailID,FNAME,LNAME,PROV,EMAIL,TEL1,ORGANIZATION
11156,11158,PAMELA,J ALLISON,NaN,PAMELA.J.ALLISON@DYNEGY.COM,NaN,DYNEGY
87472,87474,NaN,NaN,NaN,20RELATIONS@CAISO.COM,NaN,CAISO
